# Greenspace Assessment

This notebook contains code that builds on previously obtained datasets. The files required for this to run are as follows:

**Christchurch**
- greenspace shapefile: 'green_chch.shp'
- NDVI Imagery: 'ndvi_christchurch.tif'

**Melbourne**
- greenspace shapefile: 'green_melb.shp'
- NDVI Imagery: 'ndvi_melbourne.tif'

This notebook includes a series of functions that use the identified greenspace polygons as masks over a NDVI raster to draw out the median NDVI value of a park. This will help our users determine the quality of the greenspaces around them.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 66.0 MB/s eta 0:00:00


## Melbourne NDVI - vegetation health

In [9]:
import rasterio
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from rasterio.mask import mask
import matplotlib.pyplot as plt

In [ ]:
# Load the GeoTIFF (NDVI image)
ndvi_file = '/content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_melbourne.tif'
ndvi_data = rasterio.open(ndvi_file)

# Load the shapefile
file = '/content/drive/MyDrive/FinalProject/Data/Landuse-data/Melbourne/public-parks.gpkg'
gdf = gpd.read_file(file)
gdf = gdf.to_crs('EPSG:4326')

In [ ]:
park_ndvi_averages = []

for index, row in gdf.iterrows():
    park_geometry = row['geometry']
    park_name = row['name']  # Replace with the actual attribute name in your shapefile

    # Mask the NDVI image with the park's geometry
    park_ndvi, park_ndvi_transform = mask(ndvi_data, [park_geometry], crop=True)

    # Calculate the average NDVI value for the park
    average_ndvi = np.nanmean(park_ndvi)
    park_ndvi_averages.append({'Park Name': park_name, 'Average NDVI': average_ndvi})

In [ ]:
# Create a pandas dataframe from the dictionary
park_ndvis = pd.DataFrame.from_dict(park_ndvi_averages)

# Join the dataframe to the orignal geodataframe
full_gdf = gdf.join(park_ndvis)

# finding the greenspaces with an NDVI greater than 0
greenspace = full_gdf.loc[full_gdf['Average NDVI'] > 0]

# choosing only polygons
df = greenspace.loc[greenspace.geometry.type == 'Polygon']

In [ ]:
# Create high, medium, low categories
# these will help our user understand the quality of the greenspace surrounding them
# Define the conditions and categories
conditions = [
    (df['Average NDVI'] >= 0.6),
    (df['Average NDVI'] >= 0.3),
    (df['Average NDVI'] < 0.3)
]

categories = ['High', 'Medium', 'Low']

# Use np.select to categorize the values
df['Greenspace Quality'] = np.select(conditions, categories)

# close the open ndvi dataset
ndvi_data.close()

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
df.to_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Melbourne/ndvi-quality.gpkg', driver='GPKG')

## Christchurch vegetation health

In [32]:
# Load the GeoTIFF (NDVI image)
ndvi_file = '/content/drive/MyDrive/FinalProject/Data/NDVI-imagery/NDVI_christchurch.tif'
chc_ndvi_data = rasterio.open(ndvi_file)

# Load the shapefile
file = '/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/public-parks.gpkg'
gdf = gpd.read_file(file)
chc_gdf = gdf.to_crs('EPSG:4326')

In [33]:
# clip to the size of the raster to account for slightly smaller image being downloaded from gee
chc_gdf = chc_gdf.clip(chc_ndvi_data.bounds)

In [34]:
park_ndvi_averages = []

for index, row in chc_gdf.iterrows():
    park_geometry = row['geometry']
    park_name = row['name']  # Replace with the actual attribute name in your shapefile

    # Mask the NDVI image with the park's geometry
    park_ndvi, park_ndvi_transform = mask(chc_ndvi_data, [park_geometry], crop=True)

    # Calculate the average NDVI value for the park
    average_ndvi = np.nanmean(park_ndvi)
    park_ndvi_averages.append({'Park Name': park_name, 'Average_NDVI': average_ndvi})

In [35]:
# Create a pandas dataframe from the dictionary
park_ndvis = pd.DataFrame.from_dict(park_ndvi_averages)

# Join the dataframe to the orignal geodataframe
full_gdf = gdf.join(park_ndvis)

# finding the greenspaces with an NDVI greater than 0
greenspace = full_gdf.loc[full_gdf['Average_NDVI'] > 0]

# choosing only polygons
df = greenspace.loc[greenspace.geometry.type == 'Polygon']

In [37]:
# Create high, medium, low categories
# these will help our user understand the quality of the greenspace surrounding them
# Define the conditions and categories
conditions = [
    (df['Average_NDVI'] >= 0.6),
    (df['Average_NDVI'] >= 0.3),
    (df['Average_NDVI'] < 0.3)
]

categories = ['High', 'Medium', 'Low']

# Use np.select to categorize the values
df['Greenspace_Quality'] = np.select(conditions, categories)

# close the open ndvi dataset
chc_ndvi_data.close()

In [38]:
df.to_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/ndvi-quality.gpkg', driver='GPKG')